In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AWtgzh6RCr1A5pzv3m3ofURQUvnUyzBGHLoMaTPqF0htSt5LSd3oVuHgjTg

Successfully saved authorization token.


In [2]:
import geemap.foliumap as geemap

title_map = 'AOI MAP potential go zone'

#AOI
AOI_shp = './shp/AOI.shp'
AOI_shp_plot = geemap.shp_to_ee(AOI_shp)
crs_input = 'EPSG:32748'

AOI = AOI_shp_plot

#CloudLess Images

START_DATE = '2021-1-1'   #Change this as perusal
END_DATE = '2021-12-31'

#for area
OID = 'OBJECTID'

#FOR FOREST DETECTION
# Colors fo raster
water = '#3380cc' #Blue 
hi_forest = '#006837' #Strong green
low_forest = '#3ea540' #Medium green
grass_land = '#baf096' #Light green
bare_land = '#ad8855' #Brown
other = '#000000'

#COEFICIENTS: These coeficients are orientative
#and some tweak may be needed depending on the
#location and case of study

#NDWI water limit
ndwi_hi = 0.05

#Bare soil index (BI), soil limit
bi_hi = 2

#NDVI high and low limits
ndvi_lo = 0.25     #0.20 for L1C (suggested value)
                   #0.25 for L2A (suggested value)
ndvi_hi = 0.45     #0.40 for L1C (suggested value)
                   #0.45 for L2A (suggested value)

#Shadow index (SI) high and low limits
si_lo = 0.92       #0.90 for L1C (suggested value) 
                   #0.92 for L2A (suggested value)
si_hi = 0.95       #0.93 for L1C (suggested value) 
                   #0.95 for L2A (suggested value)

##################################################################################



In [3]:
#Deforestation areas
gfc = ee.Image("UMD/hansen/global_forest_change_2021_v1_9")

#Canopy cover percentage (e.g. 30%), for Indonesia
cc = ee.Number(30)

#Minimum forest area in pixels (e.g. 3 pixels, ~ 0.27 ha in this example).
pixels = ee.Number(3)

#Minimum mapping area for tree loss (usually same as the minimum forest area).
lossPixels = pixels

canopyCover = gfc.select(['treecover2000'])
canopyCover30 = canopyCover.gte(cc).selfMask()

#Use connectedPixelCount() to get contiguous area.
contArea = canopyCover30.connectedPixelCount()
#Apply the minimum area requirement.
minArea = contArea.gte(pixels).selfMask()

prj = gfc.projection()
scale = prj.nominalScale()

Map = geemap.Map(center=(-3, 115), zoom=4)
Map.centerObject(AOI, 10)
Map.addLayer(minArea.reproject(prj.atScale(scale)), {
    'palette': ['#96ED89']
}, 'tree cover: >= min canopy cover & area (light green)',False)

#create visual boundary color only
empty = ee.Image().byte()
AOIm = empty.paint(AOI,0,10)

#Map.addLayer(AOIm,{'palette': '#e043f3'},'AOI') #still buggy
#Map.addLayer(AOIm,{},'AOI')

'''
forestArea = minArea.multiply(ee.Image.pixelArea()).divide(10000)
forestSize = forestArea.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=AOI.geometry(),
    scale=30,
    maxPixels=1e13
)

print(
    'Year 2000 tree cover (ha) \nmeeting minimum canopy cover and \nforest area thresholds \n ',
    forestSize.get('treecover2000'))
    
'''

'''
pixelCount = minArea.reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=AOI.geometry(),
    scale=30,
    maxPixels=1e13
)
onePixel = forestSize.getNumber('treecover2000').divide(pixelCount.getNumber('treecover2000'))
minAreaUsed = onePixel.multiply(pixels)
print('Minimum forest area used (ha)\n ', minAreaUsed)
'''

"\npixelCount = minArea.reduceRegion(\n    reducer=ee.Reducer.count(),\n    geometry=AOI.geometry(),\n    scale=30,\n    maxPixels=1e13\n)\nonePixel = forestSize.getNumber('treecover2000').divide(pixelCount.getNumber('treecover2000'))\nminAreaUsed = onePixel.multiply(pixels)\nprint('Minimum forest area used (ha)\n ', minAreaUsed)\n"

In [4]:
treeLossYear = gfc.select(['lossyear'])
treeLoss = treeLossYear.gte(12).selfMask() # tree loss in year > 2012 ####SHOULD CHANGE TO RECENT YEAR for the '12' number
#Select the tree loss within the derived tree cover
#(>= canopy cover and area requirements).
treecoverLoss = minArea.And(treeLoss).rename('lossfrom2012').selfMask()

#Create connectedPixelCount() to get contiguous area.
contLoss = treecoverLoss.connectedPixelCount()
#Apply the minimum area requirement.
minLoss = contLoss.gte(lossPixels).selfMask()
'''
lossArea = minLoss.multiply(ee.Image.pixelArea()).divide(10000)
lossSize = lossArea.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=AOI.geometry(),
    scale=30,
    maxPixels=1e13
)
print(
    '>Year 2012 tree loss (ha) \nmeeting minimum canopy cover and \nforest area thresholds \n ',
    lossSize.get('lossfrom2012'))
'''


"\nlossArea = minLoss.multiply(ee.Image.pixelArea()).divide(10000)\nlossSize = lossArea.reduceRegion(\n    reducer=ee.Reducer.sum(),\n    geometry=AOI.geometry(),\n    scale=30,\n    maxPixels=1e13\n)\nprint(\n    '>Year 2012 tree loss (ha) \nmeeting minimum canopy cover and \nforest area thresholds \n ',\n    lossSize.get('lossfrom2012'))\n"

In [5]:
#Borrow from https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
#CLOUD_FILTER = 80
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

#s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 30})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

#s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)

# Mosaic the image collection
#img_mosaic = s2_sr_cld_col_eval_disp.mosaic()

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

SEN_BANDS = ['B2',   'B3', 'B4',  'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
bandNamesSentinel2 = ['blue', 'green', 'red', 'redE1', 'redE2', 'redE3', 'nir', 'redE4', 'swir1', 'swir2']

s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .select(SEN_BANDS, bandNamesSentinel2)
                             .median())


In [6]:
Map.addLayer(s2_sr_median.clip(AOI),
                {'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 6000, 'gamma': 1.1},
                'S2 cloud-free mosaic')

Map.addLayer(treeLossYear.clip(AOI),{"opacity":1,"bands":["lossyear"],"min":1,"max":21,"palette":["3d358c","4457c9","4777f0","4196ff","2eb4f3","1ad1d5","1ae5b6","36f493","64fd6a","92ff47","b4f836","d3e935","ecd239","fbb938","fe992c","f9751d","ec520e","d93806","bf2102","9f1001","7a0403"]},"Tree Loss Year",False)
Map.addLayer(minLoss.clip(AOI),{
    'palette': ['#ff0000']
}, 'tree cover >30% loss since 2012',False)

In [7]:
#adapted from https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/pseudo_forest_canopy_density/#
#Set to False to avoid detecting water bodies
detect_water = True
    
if detect_water:
    ndwi = s2_sr_median.normalizedDifference(['green','nir'])
    
    #Map.addLayer(ndwi,{},'NDWI')
    
    # Mask the non-watery parts of the image, where NDWI < ndwi_hi.
    waterMasked = ndwi.updateMask(ndwi.gt(ndwi_hi))
    #Map.addLayer(waterMasked, {'palette': water}, 'NDWI (Water) masked')
    
ndvi = s2_sr_median.normalizedDifference(['nir','red'])
bi_1 = s2_sr_median.expression(
    '(NIR + GREEN + RED) / (NIR + GREEN - RED)', 
        {
      'NIR': s2_sr_median.select('nir').divide(10000),
      'GREEN': s2_sr_median.select('green').divide(10000),
      'RED': s2_sr_median.select('red').divide(10000)
        })
si = s2_sr_median.expression(
     '((1 - GREEN) * (1 - RED) )**(0.5)' ,
    {
      'GREEN': s2_sr_median.select('green').divide(10000),
      'RED': s2_sr_median.select('red').divide(10000)
    })

#Map.addLayer(ndvi,{},'NDVI')
#Map.addLayer(bi_1,{},'BI_1')
#Map.addLayer(si,{},'SI')

#High density Forest
ndviHi_masked = ndvi.updateMask(ndvi.gt(ndvi_hi))
bi_1Lo_masked = bi_1.updateMask(bi_1.lt(bi_hi))
siHi_masked = si.updateMask(si.gt(si_hi))

hiforest_masked = ndviHi_masked.And(bi_1Lo_masked).And(siHi_masked)

#Create connectedPixelCount() to get contiguous area.
conthiForest = hiforest_masked.connectedPixelCount()
#Apply the minimum area requirement.
hiforest_masked = conthiForest.gte(pixels).selfMask()

Map.addLayer(hiforest_masked.clip(AOI),{'palette':hi_forest},'High Forest',False)

#Low density Forest
ndviMid_masked = ndvi.updateMask(ndvi.lt(ndvi_hi).And(ndvi.gt(ndvi_lo)))
siMid_masked = si.updateMask(si.lt(si_hi).And(si.gt(si_lo)))

loforest_masked = ndviMid_masked.And(bi_1Lo_masked).And(siMid_masked)
#Map.addLayer(loforest_masked,{'palette':low_forest},'Low Forest')

In [8]:
#Make an image out of the AOI area attribute.
AOI_img = AOI.filter(ee.Filter.notNull([OID])).reduceToImage(
    properties= [OID],
    reducer= ee.Reducer.first()
)

#Map.addLayer(AOI_img) #check if it works

HiForestAndLoss = AOI_img.And(hiforest_masked.And(minLoss))
#Map.addLayer(HiForestAndLoss,{'palette': '#FF0000'},"High Baseline and not pass 10 years rule")

#Create a mask indicating where the smaller areas (minLoss)
maskHiFL = HiForestAndLoss.mask()

#Invert the mask to get the areas where the smaller raster is absent
maskHiFL_inverted = maskHiFL.Not()

#Unmask the bigger raster in the areas where the smaller raster is absent
unmaskedHiFL = AOI_img.unmask().updateMask(maskHiFL_inverted).clip(AOI)

tenYearsRule = unmaskedHiFL.And(minLoss)

waterinAOI = waterMasked.And(AOI_img)

#Map.addLayer(tenYearsRule,{'palette': '#FFA500'},"10 Years Rule - not OK")

#Map.addLayer(waterinAOI,{'palette': water},"Water in AOI")

#Similar with above, unmasked the high forest
maskHiF = hiforest_masked.mask()
maskHiF_inverted = maskHiF.Not()
unmaskedHiF = AOI_img.unmask().updateMask(maskHiF_inverted).clip(AOI)

#Unmasked the forest loss 10 years rule
unmaskedLoss = AOI_img.unmask().updateMask(minLoss.mask().Not()).clip(AOI)

#unmasked the water
unmaskedWaterAOI = AOI_img.unmask().updateMask(waterinAOI.mask().Not()).clip(AOI)

highBaselineF = hiforest_masked.And(unmaskedLoss)

goZone = unmaskedLoss.And(unmaskedHiF).And(unmaskedWaterAOI)

#Map.addLayer(goZone,{'palette':'#FFFF00'},'Go Zone')
#Map.addLayer(highBaselineF,{'palette':hi_forest},'High baseline (Forest)')


In [9]:
labels = ['High Baseline (Forest)', '10 Years Rule not OK', 'High Baseline not passed 10 years rule', 'Water', 'Go Zone']
# colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
colors = ['#006837', '#FFA500', '#FF0000', '#3380cc', '#FFFF00']
# legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]

Map.add_legend(title='Legend', labels=labels, colors=colors)

In [10]:
def assigning_band(class_value,srcImg):
        
    # Create an image with the constant value for class
    constant_image_class = srcImg.multiply(0).add(class_value).rename('Class')
    constant_image_pixel = srcImg.multiply(0).add(1).rename('pixel')
    
    # Add the new band to the existing image
    pixel_bandimg= srcImg.addBands(constant_image_pixel)
    pix_classImg = pixel_bandimg.addBands(constant_image_class)
    pix_classImg = pix_classImg.select(['Class','pixel'])
    
    return pix_classImg


goZone_edited = ee.Image(assigning_band(1,goZone)) #go Zone - 1
highf_edited = ee.Image(assigning_band(2,highBaselineF)) #high baseline forest - 2
tenyrfl_edited = ee.Image(assigning_band(3,HiForestAndLoss)) #tenyears rule not pass and high baseline - 3
tenyrule_edited = ee.Image(assigning_band(4,tenYearsRule)) #tenyears rule not pass - 4
water_edited = ee.Image(assigning_band(5,waterinAOI))

#image_list = ee.List([goZone_edited, highf_edited, tenyrfl_edited, tenyrule_edited, water_edited ])
# Extract the spatial information from one of the individual images.
projection = goZone_edited.projection()
scale = goZone_edited.projection().nominalScale()

# Create an ee.ImageCollection from the list of images
image_collection = ee.ImageCollection([goZone_edited, highf_edited, tenyrfl_edited, tenyrule_edited, water_edited ])

# Merge the images into a single ee.Image
zoning_final = image_collection.mosaic()

In [11]:
Map.addLayer(goZone_edited,{'bands': ['Class'],'palette':'#FFFF00'},'Go Zone')
Map.addLayer(highf_edited,{'bands': ['Class'],'palette':hi_forest},'High baseline (Forest)')
Map.addLayer(tenyrfl_edited,{'bands': ['Class'],'palette': '#FF0000'},"High Baseline and not pass 10 years rule")
Map.addLayer(tenyrule_edited,{'bands': ['Class'],'palette': '#FFA500'},"10 Years Rule - not OK")
Map.addLayer(water_edited,{'bands': ['Class'],'palette': water},"Water in AOI")

In [12]:
#vis_params = {'bands': ['Class'], 'palette': ['ab2b2b', ' 93d2a3', ' af2f18', ' 655b95', ' f3ed39'], 'min': 0.0, 'max': 5, 'opacity': 1.0}
#Map.addLayer(zoning_final,vis_params,'Project Zone')
#Map.addLayer(goZone_edited,{},'gozoneedit')
#Map.addLayer(highf_edited,{},'highfedit')

In [13]:
Map.addLayerControl()
Map

In [14]:
#png_file = './my_map.png'
#Map.to_image(filename=png_file, monitor=1)

html_file = f'./{title_map}.html'
Map.to_html(filename=html_file, title=title_map)

In [15]:
# Create a list of images to analyze
list_images = [goZone_edited, highf_edited, tenyrfl_edited, tenyrule_edited, water_edited]

# Define a function to calculate the area of an image within the AOI
def areas_calculation(img):
    # Calculate the total number of pixels in the ROI
    area2 = img.reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry=AOI,
      crs=crs_input,
      scale=30,
      maxPixels=1e13
    )
    
    print('area calculated \n ------')

    return ee.Number(area2.get('pixel')).getInfo()

# Select the 'pixel' band and calculate the area of each pixel in hectares
list_areas = [areas_calculation(image.select(['pixel']).multiply(ee.Image.pixelArea()).divide(10000)) for image in list_images]

# Map the areas_calculation function over the list of images to calculate the area of each image
#list_areas = ee.List(selected_band_list_images).map(areas_calculation)

area calculated 
 ------
area calculated 
 ------
area calculated 
 ------
area calculated 
 ------
area calculated 
 ------


In [16]:
list_areas

[150.9305549999997,
 299.0448921755793,
 0.08988359985351563,
 0.8089479858398436,
 0]

In [17]:
print(list_areas)

[150.9305549999997, 299.0448921755793, 0.08988359985351563, 0.8089479858398436, 0]


In [18]:
import pandas as pd
data = {'Class': [1, 2, 3, 4,5], 
        'Name_Class': ['Go Zone', 'High Forest Baseline', 'High Baseline no pass 10 years rule', 
                       'No pass 10 years rule', 'Water (Un-plantable)'],
        'Area_Ha':list_areas}
pd.DataFrame.from_dict(data)

,Class,Name_Class,Area_Ha
0,1,Go Zone,150.930555
1,2,High Forest Baseline,299.044892
2,3,High Baseline no pass 10 years rule,0.089884
3,4,No pass 10 years rule,0.808948
4,5,Water (Un-plantable),0.000000


In [ ]:
'''
goZoneArea = goZone.Not().multiply(ee.Image.pixelArea()).divide(10000) #make the zero number into 1 so that we need '.Not()'

# Calculate the total number of pixels in the ROI
area2 = goZoneArea.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=AOI,
  crs = 'EPSG:32650',
  scale=30,
  maxPixels=1e13
)
 
goZoneAreaHa = ee.Number(
  area2.get('first')).getInfo()
print(goZoneAreaHa)

highBaselineF_Area = highBaselineF.Not().multiply(ee.Image.pixelArea()).divide(10000) #make the zero number into 1 so that we need '.Not()'

# Calculate the total number of pixels in the ROI
area2_hBF = highBaselineF_Area.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=AOI,
  crs = 'EPSG:32650',
  scale=30,
  maxPixels=1e13
)
 
highBaselineF_AreaHa = ee.Number(
  area2_hBF.get('nd')).getInfo()
print(highBaselineF_AreaHa)

waterinAOI_Area = waterinAOI.Not().multiply(ee.Image.pixelArea()).divide(10000) #make the zero number into 1 so that we need '.Not()'

# Calculate the total number of pixels in the ROI
area2_wtr = waterinAOI_Area.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=AOI,
  crs = 'EPSG:32650',
  scale=30,
  maxPixels=1e13
)
 
waterinAOI_AreaHa = ee.Number(
  area2_wtr.get('nd')).getInfo()
print(waterinAOI_AreaHa)

HiForestAndLoss_Area = HiForestAndLoss.Not().multiply(ee.Image.pixelArea()).divide(10000) #make the zero number into 1 so that we need '.Not()'

# Calculate the total number of pixels in the ROI
area2_hifl = HiForestAndLoss_Area.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=AOI,
  crs = 'EPSG:32650',
  scale=30,
  maxPixels=1e13
)
 
HiForestAndLoss_AreaHa = ee.Number(
  area2_hifl.get('first')).getInfo()
print(HiForestAndLoss_AreaHa)

tenYearsRule_Area = tenYearsRule.Not().multiply(ee.Image.pixelArea()).divide(10000) #make the zero number into 1 so that we need '.Not()'

# Calculate the total number of pixels in the ROI
area2_10yrule = tenYearsRule_Area.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=AOI,
  crs = 'EPSG:32650',
  scale=30,
  maxPixels=1e13
)
 
tenYearsRule_AreaHa = ee.Number(
  area2_10yrule.get('first')).getInfo()
print(tenYearsRule_AreaHa)
'''

In [ ]:
'''
zoning_cls = zoning_final.select(['Class'])
zoning_pixel = zoning_final.select(['pixel']).multiply(ee.Image.pixelArea()).divide(10000)

zoning_Area_cls = zoning_pixel.addBands(zoning_cls).reduceRegion(
  reducer=ee.Reducer.sum().group(groupField=1),
  geometry=AOI,
  scale=30,
  maxPixels=1e13
  )

statsFormatted = ee.List(zoning_Area_cls.get('groups')) \
  .map(lambda el: ee.Dictionary(el)) \
  .map(lambda d: [ee.Number(d.get('group')), d.get('sum')])

statsDictionary = ee.Dictionary(statsFormatted.flatten())
print(statsDictionary)
'''